# Volatility

---

In finance, volatility is the magnitude of an asset’s price or return fluctuations over time, typically quantified as the standard deviation of returns and used as a practical proxy for risk. A highly volatile asset exhibits large, frequent deviations from its average return, while a low‑volatility asset tends to move in smaller, more clustered steps. 

Within this broad concept, it is useful to distinguish between different flavours of volatility. 
* Realized volatility is a backward‑looking, purely statistical measure computed from historical returns over a chosen window (for example, the annualized standard deviation of the last 21 daily log returns). 
* Implied volatility, by contrast, is a forward‑looking quantity inferred from option prices: it is the volatility input that makes a pricing model (such as Black–Scholes) reproduce observed option premiums and therefore embeds the market’s expectation of future variability plus any volatility risk premium. 

Much of options trading and risk management revolves around comparing these two “views” of the same underlying uncertainty: the volatility the market is pricing in today versus the volatility that has actually been realized in the past and is expected to realize going forward.


We can classify volatility modelling into three broad categories. The point of splitting them this way is to make clear what you are modelling (realized paths vs. priced risk) and what decision each model supports: forecasting risk, pricing derivatives, or arbitraging the gap between the two. In other words, you want a clean pipeline from beliefs about the real world to risk‑neutral prices and then to trades that are explicitly about the implied-realized volatility spread.

1. **Models of realized volatility (physical, ex post)**

These start from historical returns and model the path of realized variance under the physical measure $\mathbb{P}$. Let
$$
r_t = \ln\left(\frac{S_t}{S_{t-1}}\right)
$$
denote log‑returns, and let the conditional variance be $\sigma_t^2$.

- ARCH/GARCH family (e.g. GARCH(1,1)):
$$
\begin{aligned}
r_t &= \mu + \varepsilon_t, \quad \varepsilon_t \sim (0, \sigma_t^2), \\\\
\sigma_t^2 &= \omega + \alpha \varepsilon_{t-1}^2 + \beta \sigma_{t-1}^2.
\end{aligned}
$$

- Realized‑volatility / HAR‑RV models work with realized variance $RV_t$ constructed from high‑frequency data and then specify a time‑series model such as
$$
RV_t = \beta_0 + \beta_d RV_{t-1} + \beta_w RV_{t-5:t-1} + \beta_m RV_{t-22:t-1} + u_t,
$$
where $RV_{t-5:t-1}$ and $RV_{t-22:t-1}$ denote weekly and monthly averages.

These models answer questions of the form
$$
\mathbb{E}_{\mathbb{P}}\big[\sigma_{t+h}^2 \mid \mathcal{F}_t\big],
$$
i.e., forecasts of future realized variance conditional on the information set $\mathcal{F}_t$.

**Goal:** produce conditional forecasts of volatility for risk limits, leverage, volatility targeting, and PnL distribution analysis in the real world.

2. **Models of implied volatility (risk‑neutral, pricing)**

These are calibrated to option prices and describe the implied volatility surface under the risk‑neutral measure $\mathbb{Q}$.

- Local volatility models specify an instantaneous volatility
$$
\sigma_{\text{loc}} = \sigma_{\text{loc}}(S_t, t)
$$
so that the associated diffusion
$$
dS_t = r S_t \, dt + \sigma_{\text{loc}}(S_t, t) S_t \, dW_t^{\mathbb{Q}}
$$
matches all observed option prices at time $t$.

- Stochastic volatility models (e.g. Heston) specify
$$
\begin{aligned}
dS_t &= r S_t \, dt + \sqrt{v_t} S_t \, dW_t^{(1),\mathbb{Q}}, \\\\
dv_t &= \kappa^{\mathbb{Q}}(\theta^{\mathbb{Q}} - v_t) \, dt + \xi \sqrt{v_t} \, dW_t^{(2),\mathbb{Q}},
\end{aligned}
$$
with $\text{corr}(dW_t^{(1),\mathbb{Q}}, dW_t^{(2),\mathbb{Q}}) = \rho$, and parameters chosen to fit the implied volatility smile/skew and term structure.

Parametric IV surface models (e.g. SVI) directly specify a functional form
$$
\sigma_{\text{imp}}(k, T)
$$
for implied volatility as a function of log‑moneyness $k$ and maturity $T$, fitted to option data.

These models answer: under $\mathbb{Q}$, what volatility surface is consistent with observed option prices?

**Goal:** build a no‑arbitrage volatility surface and pricing engine so options, swaps and exotics are valued consistently and hedges (Greeks) are comparable across strikes and maturities.

3. **Joint return / volatility process models (linking $\mathbb{P}$ and $\mathbb{Q}$)**

These specify a full data‑generating process under the physical measure $\mathbb{P}$ and then relate it to pricing under $\mathbb{Q}$ via a change of measure.

- Physical‑measure stochastic volatility:
$$
\begin{aligned}
dS_t &= \mu S_t \, dt + \sqrt{v_t} S_t \, dW_t^{(1),\mathbb{P}}, \\\\
dv_t &= \kappa(\theta - v_t) \, dt + \xi \sqrt{v_t} \, dW_t^{(2),\mathbb{P}}.
\end{aligned}
$$
A change of measure $\mathbb{P} \to \mathbb{Q}$ introduces volatility risk premia in the drift of $v_t$, linking realized and implied volatility.

- Jump‑diffusion with stochastic volatility:
$$
dS_t = \mu S_t \, dt + \sqrt{v_t} S_t \, dW_t^{(1),\mathbb{P}} + S_{t^-} \, dJ_t,
$$
where $J_t$ is a jump process (e.g. compound Poisson) and $v_t$ follows its own stochastic dynamics.

These joint models generate both realized volatility paths under $\mathbb{P}$ and option prices (and hence implied volatility) under $\mathbb{Q}$.

**Goal:** connect “what I think will happen” to “what should be priced,” quantify the volatility risk premium, and design trades that exploit misalignments between implied and expected realized volatility in a theoretically coherent way.
